<a href="https://colab.research.google.com/github/CharlotteY2003/MIT-Futuremakers/blob/main/7_16_MNSIT_Classification_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import mean, std
import pandas as pd
from keras.preprocessing.image import load_img
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import *
from keras.datasets import mnist
from sklearn.model_selection import train_test_split, KFold
from keras.utils.all_utils import to_categorical
from keras.optimizers import SGD

In [ ]:
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
#for num in range(9):
#  plt.subplot(330+1+num)
#  plt.imshow(X_train[num], cmap = plt.get_cmap('gray'))
#  plt.xticks([])
#  plt.yticks([])


#plt.show()

In [ ]:
#print(X_train.shape, X_test.shape)
#print(y_train.shape, y_test.shape)

In [ ]:
def load_data(classes=10):
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
  X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
  y_train = to_categorical(y_train, num_classes = classes)
  y_test = to_categorical(y_test, num_classes=classes)
  return X_train, y_train, X_test, y_test

In [ ]:
def prepare_data(train,test):
  #Convert unassigned int to double
  train = train.astype('float32')
  test = test.astype('float32')
  #Normalize to range 0-1
  train = train / 255.0
  test = test / 255.0
  #Return results
  return train, test

In [ ]:
def define_model():
  model = Sequential()
  model.add(Conv2D(filters= 32, kernel_size= (3,3), input_shape=(28,28,1), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPool2D())
  model.add(Conv2D(filters = 64, kernel_size = (3,3), input_shape = (28,28,1), activation = 'relu', kernel_initializer='he_uniform'))
  model.add(Conv2D(filters = 64, kernel_size = (3,3), input_shape = (28,28,1), activation = 'relu', kernel_initializer='he_uniform'))
  model.add(MaxPool2D())
  model.add(Flatten())
  model.add(Dense(units=100, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(units = 10, activation='softmax', kernel_initializer='he_uniform'))

  optimizer = SGD(lr=.01, momentum = .9)
  model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

In [ ]:
def evaluate_model(model, datax, datay, nfolds=5):
  scores, histories = list(), list()
  kf = KFold(nfolds, shuffle=True, random_state=1)
  
  for trainix, testix in kf.split(datax):
    model = define_model()
    X_train, y_train, X_test, y_test = datax[trainix], datay[trainix], datax[testix], datay[testix]
    history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data = (X_test, y_test), verbose=0)
    _, acc = model.evaluate(X_test, y_test, verbose=0) #Confused - why evaluate on data model has already seen?
    print(acc)                                         #explanation = some ML models only have train + val set -> need to save actual test data for final model
    scores.append(acc)
    histories.append(history)
  return scores, histories

In [ ]:
def summarize_diagnosis(histories):
  print(len(histories))
  for i in range(len(histories)):
    plt.subplot(2,1,1)
    plt.title('Cross Entropy Loss')
    plt.plot(histories[i].history['loss'], color = 'blue', label = 'train')
    plt.plot(histories[i].history['val_loss'], color = 'red', label = 'test')
    plt.subplot(2,1,2)
    plt.title('Model Accuracy')
    plt.plot(histories[i].history['accuracy'], color = 'blue', label = 'train')
    plt.plot(histories[i].history['val_accuracy'], color = 'red', label = 'test')
  plt.show()

In [ ]:
def summarize_performance(scores):
  print('Mean: ' + str(mean(scores)))
  print('Standard deviation: ' + str(std(scores)))
  plt.boxplot(scores)
  plt.show()

In [ ]:
def run_test_harness():
  X_train, y_train, X_test, y_test = load_data()
  X_train, X_test = prepare_data(X_train, X_test)
  model = load_model('final_model.h5')
  _, acc = model.evaluate(X_test, y_test, verbose=0)
  print(acc)
  #model = define_model()
  #model.fit(X_train, y_train, epochs=10, batch_size = 32, verbose=0)
  #model.save('final_model.h5')
  #scores, histories = evaluate_model(model, X_train, y_train)
  #summarize_diagnosis(histories)
  #summarize_performance(scores)

In [ ]:
#run_test_harness()

In [ ]:
def load_image(filename):
  image = load_img(filename,grayscale=True, target_size = (28,28))
  image = np.array(image)
  image = image.reshape(1, 28,28,1)
  image = image.astype('float32')
  image = image/255.0
  return image

In [ ]:
def run_example():
  img = load_image('sample_image.png')
  model = load_model('final_model.h5')
  digit = model.predict_classes(img)
  print(digit)

In [ ]:
run_example()

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[7]
